In [14]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [8]:
data = pd.read_csv('words.csv', sep='\t', names=['word', 'valid'])

In [37]:
target = data.valid.astype(int).values

In [38]:
target

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
print len(data[data.valid == True])
print len(data[data.valid == False])

50703
52283


In [28]:
print len(data)
print len(data.word.unique())

102986
102986


In [111]:
from sklearn.feature_extraction.text import CountVectorizer

ngram_vectorizer = CountVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 6),
    min_df=1,
    max_features=250000
) 

ngram_data = ngram_vectorizer.fit_transform(data.word)

In [112]:
ngram_data.shape

(102986, 250000)

In [113]:
from sklearn.cross_validation import train_test_split

xtr, xcv, ytr, ycv = train_test_split(ngram_data, target, test_size = 0.05,  random_state = 42)

In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

cv = 3
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}

lr = LogisticRegression(
    penalty='l2', # 'l1'
    solver='lbfgs',  # 'newton-cg', 'lbfgs', 'liblinear', 'sag'
    fit_intercept=False,
    multi_class='ovr'
)

gc = GridSearchCV(lr, param_grid, cv=cv)
gc.fit(xtr, ytr)

GridSearchCV(cv=3, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [60]:
gc.best_estimator_

LogisticRegression(C=0.05, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [124]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(
    C=0.05,
    penalty='l2', # 'l1'
    solver='lbfgs',  # 'newton-cg', 'lbfgs', 'liblinear', 'sag'
    fit_intercept=False,
    multi_class='ovr'
)

lr.fit(xtr, ytr)
predictions = lr.predict_proba(xcv)

In [70]:
# predictions = gc.best_estimator_.predict_proba(xcv)

In [125]:
from sklearn.metrics import roc_auc_score

roc_auc_score(ycv, predictions[:,1])

0.71582251897340832

The best: 0.71593944320529912

In [107]:
from sklearn.cross_validation import cross_val_score

cross_val_score(lr, ngram_data, target, cv=3).mean()

0.54412250445313604